In [1]:
import os
# necessary when on deepnet server
os.environ['DISPLAY'] = ':0.0'

import numpy as np
from PIL import Image
from GFG import Ctx
from GFG.model import Nf
from pathlib import Path
from GFG.core import Camera
from skimage import transform
from GFG.model import Furmap, hcol, Adata
from GFG.identity import IDModel
import sys
sys.path.append('../scripts/generate_stimuli')
from generate_backgrounds import phase_scramble_image

Made context-safe: Ebo bind <class 'function'>
Made context-safe: Ebo delete <class 'function'>
Made context-safe: Vbo bind <class 'function'>
Made context-safe: Vbo update <class 'function'>
Made context-safe: Vbo delete <class 'function'>
Made context-safe: Texture create_texture2D <class 'function'>
Made context-safe: Texture create_texture2DMS <class 'function'>
Made context-safe: Texture create_texture2Darr <class 'function'>
Made context-safe: Texture set_as_active <class 'function'>
Made context-safe: Texture delete <class 'function'>
Made context-safe: Renderbuffer create_renderbuffer <class 'function'>
Made context-safe: Renderbuffer create_renderbufferMS <class 'function'>
Made context-safe: Renderbuffer delete <class 'function'>
Made context-safe: Fbo bind <class 'function'>
Made context-safe: Fbo initialise <class 'function'>
Made context-safe: Fbo assert_complete <class 'function'>
Made context-safe: Fbo delete <class 'function'>
Made context-safe: ShaderProgram update_uni

In [ ]:
base_nf = Nf.from_default()
furmap = Furmap(hcol=hcol['Dark Golden Brown'], nlayers=120, density=0.15)
furmap.material.split_newmtl(mat_groups = base_nf.material_groups,
                             fvt = base_nf.face_tuple.fv,
                             vt = base_nf.vertex_tuple.dSvt,
                             groupfidx = base_nf.groupfindex)
base_nf.attach_fur(furmap, name='fur')

### SETUP CAMERA + DEFINITION PARAMETERS
# Setup openGL context + camera
ctx = Ctx(hidden=True)
base_nf.attach(ctx)  # attach to openGL context
ctx._camera[0] = Camera(
    ctx.win, (512, 512), 4.,  # res, renderscale
    target=[-11.5644, -13.0381, 0],
    # Increase distance to 700 (fits face in FOV)
    eye = [-11.5644, -13.0381, 350],
    up = [0, 1, 0],
    FOV = 50,
    near = 100.,
    far = 1000.
)
#ctx.framebuffer_from_camera(ctx._camera[0])
ctx.assign_camera(0)

In [ ]:
ctx.set_lights('flat')
base_nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
img = ctx.render('image')
img.save('../data/orig.png')
img

In [ ]:
base_nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
base_nf.transform_model(15, 25, 0, [40, -10, 0], order='xyzt', replace=False)
img = ctx.render('image')
img.save('../data/rot.png')
img

In [ ]:
base_nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
base_nf.transform_model(15, 25, 0, [0, 0, 0], order='xyzt', replace=False)
img = ctx.render('image')

img_arr = np.array(img)
tform = transform.EuclideanTransform(translation=(120, 80))
img_arr = transform.warp(img_arr, tform.inverse)
img_arr *= 255
img = Image.fromarray(img_arr.astype(np.uint8))
img.convert('RGBA').save(str('../data/trans+rot.png'))
img


In [ ]:
ctx.set_lights(Path('../lights.yaml'))
ctx.transform_lights(45, 45, 0, [0, 0, 0])

base_nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
base_nf.transform_model(15, 25, 0, [0, 0, 0], order='xyzt', replace=False)
img = ctx.render('image')

img_arr = np.array(img)
tform = transform.EuclideanTransform(translation=(120, 80))
img_arr = transform.warp(img_arr, tform.inverse)
img_arr *= 255
img = Image.fromarray(img_arr.astype(np.uint8))
img.convert('RGBA').save(str('../data/trans+rot+lights.png'))
img

In [ ]:
ctx.set_lights(Path('../lights.yaml'))
ctx.transform_lights(45, 45, 45, [0, 0, 0])

base_nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
base_nf.transform_model(15, 25, 0, [0, 0, 0], order='xyzt', replace=False)
img = ctx.render('image')

img_arr = np.array(img)
tform = transform.EuclideanTransform(translation=(120, 80))
img_arr = transform.warp(img_arr, tform.inverse)
img_arr *= 255
bg_rgb = phase_scramble_image(img_arr.copy(), None, is_image=False, shuffle_phase=False, grayscale=False)
img_rgb, img_a = img_arr[..., :3], img_arr[..., 3, None] / 255.
img_arr = (img_rgb * img_a) + (bg_rgb * (1 - img_a))  # alpha blend
img_arr = np.dstack((img_arr, np.ones((512, 512)) * 255)).astype(np.uint8)
img = Image.fromarray(img_arr.astype(np.uint8))
img.convert('RGBA').save(str('../data/trans+rot+lights+bg.png'))
bg = Image.fromarray(bg_rgb.astype(np.uint8))
bg.convert('RGBA').save('../data/orig_background.png')